Installing required packages 

In [55]:
%pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


Importing required packages

In [56]:
import pandas as pd
from io import StringIO
import json
from sklearn.preprocessing import LabelEncoder


Creating LabelEncoder object for use within our main function

In [57]:
le = LabelEncoder()

Creating two sets to make sure we only include **active players** in the final CSV

In [58]:
with open("data/players.json") as file:
    players = json.load(file)

active_players_name = set()
active_players_code = set()

for player in players.values():
    active_players_name.add(player['Player'])
    active_players_code.add(player['PlayerCode'])

Function to update each row in our new dataframe with the defensive ratings (`opp_def_rtg` and `opp_def_rtg_adj`)

In [59]:
def get_def_ratings(row):
    with open('data/def_rating.json', 'r') as file:
        def_rtgs = json.load(file)
    team_def_rtg = def_rtgs[row['Opponent'].upper()]
    year = int(row['GameDay'].split("-")[0])
    for i in team_def_rtg:
        if i['year'] == year:
            return i['def_rtg'], i['def_rtg_adj']
    
    return -1, -1


Main function to generate and return a dataframe for each year specified, as well as player and team mappings for later analysis

In [60]:
def generate_dataframe_by_year(year):
    print(f"Processing year: {year}")
    # Open the CSV file for the specific year and read its contents
    with open(f"data/NatStat-NBA{year}-Player_Statlines-2024-09-17-h13.csv", "r") as file:
        csv_content = file.read()

    # Use StringIO to treat the CSV content as a file-like object for reading into pandas
    data = StringIO(csv_content)

    # Read the CSV into a pandas DataFrame, treating all columns as strings
    df = pd.read_csv(data, dtype={"GameDay": "string", "GameID" : "string", "Player" : "string", "PlayerID" : "string", 
                                  "PlayerCode": "string", "TeamID" : "string", "Team" : "string", "OpponentID": "string", 
                                  "Opponent" : "string", "Location" : "string", "Division" : "string", "Conference" : "string", 
                                  "Playoffs" : "string", "WinOrLoss" : "string", "Starter" : "string", "PlayerType" : "string", 
                                  "PerfScore" : "string", "MIN" : "string", "PTS" : "string", "FGM" : "string", "FGA" : "string", 
                                  "3FM" : "string", "3FA" : "string", "FTM" : "string", "FTA" : "string", "REB" : "string", 
                                  "AST" : "string", "STL" : "string", "BLK" : "string", "OREB" : "string", "TO" : "string", 
                                  "PF" : "string"})

    df_raw = df.copy()
    # Check if 'PlayerCode' exists, if not, create the column and fill it with 0 as in some years this column doesn't exist
    if 'PlayerCode' not in df.columns:
        df['PlayerCode'] = 0

    # Apply the get_def_ratings function to get opponent defensive ratings and add them to the DataFrame
    df[['opp_def_rtg', 'opp_def_rtg_adj']] = df.apply(get_def_ratings, axis=1, result_type="expand")

    # Filter the DataFrame to keep only active players based on PlayerCode or Player name
    df = df[(df['PlayerCode'].isin(active_players_code)) | (df['Player'].isin(active_players_name))]

    # Create dictionaries mapping PlayerID to Player name and TeamID to Team name
    current_player_mappings = df.set_index('PlayerID')['Player'].to_dict()
    current_team_mappings = df.set_index('TeamID')['Team'].to_dict()

    # Drop unnecessary columns from the DataFrame
    df.drop(columns=['GameDay', 'Player', 'PlayerCode', 'Team', 'Opponent', 'Division', 'Conference'], inplace=True)

    # Fill missing values (NaNs) with '0'
    df.fillna('0', inplace=True)

    # Label encode categorical columns
    df['Location'] = le.fit_transform(df['Location'])
    df['Playoffs'] = le.fit_transform(df['Playoffs'])
    df['WinOrLoss'] = le.fit_transform(df['WinOrLoss'])
    df['Starter'] = le.fit_transform(df['Starter'])

    # Convert all columns to numeric, replacing non-numeric values with 0
    df = df.apply(pd.to_numeric, errors="coerce").fillna(0)

    # Print a message indicating the processing of the year is finished
    print(f"Finished year: {year}")
    
    # Return the modified DataFrame and the current player and team mappings
    return df, current_player_mappings, current_team_mappings, df_raw

In [61]:
final = pd.DataFrame()
final_raw = pd.DataFrame()
player_mappings = {}
team_mappings = {}
years = [2000 + i for i in range(4, 25)]

for year in years:
    # Call function to get df and mappings
    df, current_player_mappings, current_team_mappings, df_raw = generate_dataframe_by_year(year)
    # Append the new df with our final one that we are going to save as CSV
    final = pd.concat([final, df], ignore_index=True)
    final_raw = pd.concat([final_raw, df_raw], ignore_index=True)
    # Update the two mappings with (potentially) new values
    player_mappings.update(current_player_mappings)
    team_mappings.update(current_team_mappings)

Processing year: 2004
Finished year: 2004
Processing year: 2005
Finished year: 2005
Processing year: 2006
Finished year: 2006
Processing year: 2007
Finished year: 2007
Processing year: 2008
Finished year: 2008
Processing year: 2009
Finished year: 2009
Processing year: 2010
Finished year: 2010
Processing year: 2011
Finished year: 2011
Processing year: 2012
Finished year: 2012
Processing year: 2013
Finished year: 2013
Processing year: 2014
Finished year: 2014
Processing year: 2015
Finished year: 2015
Processing year: 2016
Finished year: 2016
Processing year: 2017
Finished year: 2017
Processing year: 2018
Finished year: 2018
Processing year: 2019
Finished year: 2019
Processing year: 2020
Finished year: 2020
Processing year: 2021
Finished year: 2021
Processing year: 2022
Finished year: 2022
Processing year: 2023
Finished year: 2023
Processing year: 2024
Finished year: 2024


Write our player and tean mappings to JSON files for later use, save the final csv for later

In [127]:
with open("data/player_mappings.json", "w") as file:
    json.dump(player_mappings, file)
with open("data/team_mappings.json", "w") as file:
    json.dump(team_mappings, file)

,GameDay,GameID,Player,PlayerID,PlayerCode,TeamID,Team,OpponentID,Opponent,Location,...,3FA,FTM,FTA,REB,AST,STL,BLK,OREB,TO,PF
0,2003-10-28,16549,Antoine Walker,4014,antoine-walker,23,Dallas,27,L.A. Lakers,V,...,6,0,0,7,1,0,0,2,2,3
1,2003-10-28,16549,Josh Howard,1770,josh-howard,23,Dallas,27,L.A. Lakers,V,...,0,0,0,3,1,0,0,0,0,0
2,2003-10-28,16549,Antawn Jamison,1874,antawn-jamison,23,Dallas,27,L.A. Lakers,V,...,1,2,4,6,1,0,0,3,0,2
3,2003-10-28,16549,Tony Delk,924,tony-delk,23,Dallas,27,L.A. Lakers,V,...,3,2,2,6,1,0,0,1,0,3
4,2003-10-28,16549,Eduardo Najera,2787,eduardo-najera,23,Dallas,27,L.A. Lakers,V,...,0,0,0,3,0,1,0,1,0,2


In [129]:
with open("data/player_mappings.json", "r") as file:
    player_dict = json.load(file)

value_counts = {}
for key, value in player_dict.items():
    if value in value_counts:
        value_counts[value].append(key)
    else:
        value_counts[value] = [key]

# Step 2: Find the keys with duplicate values
duplicate_players = {value: keys for value, keys in value_counts.items() if len(keys) > 1}
final.describe()

# for k, i in duplicate_players.items():
#     z = final_raw.loc[final_raw['Player'] == k]
#     print(z)

,GameID,PlayerID,TeamID,OpponentID,Location,Playoffs,WinOrLoss,Starter,PlayerType,PerfScore,...,FTA,REB,AST,STL,BLK,OREB,TO,PF,opp_def_rtg,opp_def_rtg_adj
count,1.726110e+05,1.726110e+05,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000,172611.0,172611.000000,...,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000,172611.000000
mean,8.473649e+05,5.608076e+06,17.285046,17.174097,0.498943,0.065969,0.508971,0.581481,0.0,5.846748,...,2.643314,4.688253,2.682813,0.825434,0.617666,1.086860,1.367369,2.030473,111.638388,111.642294
std,3.897542e+05,1.386850e+07,10.505160,10.454508,0.500000,0.248229,0.499921,0.493318,0.0,4.243171,...,3.148371,3.637590,2.776356,1.025541,0.996486,1.439277,1.491016,1.474821,4.209333,4.190367
min,1.656000e+04,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,9.816230e+05,1.871000e+03,8.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.000000,...,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,108.670000,108.660000
50%,1.035028e+06,4.055000e+03,17.000000,17.000000,0.000000,0.000000,1.000000,1.000000,0.0,5.000000,...,2.000000,4.000000,2.000000,1.000000,0.000000,1.000000,1.000000,2.000000,112.030000,112.000000
75%,1.076057e+06,6.736510e+05,25.000000,25.000000,1.000000,0.000000,1.000000,1.000000,0.0,9.000000,...,4.000000,7.000000,4.000000,1.000000,1.000000,2.000000,2.000000,3.000000,114.510000,114.560000
max,1.079270e+06,5.716896e+07,38.000000,38.000000,1.000000,1.000000,1.000000,1.000000,0.0,20.000000,...,36.000000,31.000000,24.000000,10.000000,12.000000,15.000000,12.000000,7.000000,121.390000,121.180000


Normalizing all of the duplicate players and IDs (TODO)

In [ ]:
final.to_csv('data/out.csv', index=False)
final_raw.to_csv('data/out_raw.csv', index=False)